In [ ]:
!pip install pytchat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
import pytchat
import pandas as pd
import time

# ✅ Replace this with your YouTube livestream video ID
video_id = "AJ53jNaA5Fo"

# ✅ Maximum number of comments you want to collect
max_comments = 500

# Create the chat object
chat = pytchat.create(video_id=video_id)
comments = []

print("⏳ Fetching live comments... You can stop anytime by pressing the stop button above.\n")

try:
    while chat.is_alive() and (max_comments is None or len(comments) < max_comments):
        for c in chat.get().sync_items():
            comment = {
                "User ID": c.author.channelId,
                "Comment": c.message
            }
            comments.append(comment)
            print(f"{c.author.name}: {c.message}")

            if max_comments is not None and len(comments) >= max_comments:
                break
        time.sleep(1)

except KeyboardInterrupt:
    print("\n🛑 Stopped manually.")

# ✅ Save the comments to a CSV file
df = pd.DataFrame(comments)
df.to_csv("yt_live_chat.csv", index=False, encoding="utf-8")

print("\n✅ Saved", len(comments), "comments to 'yt_live_chat.csv'")

⏳ Fetching live comments... You can stop anytime by pressing the stop button above.

sol cutta: I bet most of us watching have seen all these.
ZombieAteMyPC: :alien::alien_monster:
sol cutta: love rewatch.
sol cutta: this one been on a lot
Juliana Elise 🦋: every episode :grinning_face_with_sweat:except I still gotta watch the one that premiered tonight
sol cutta: tonight's was good :shushing_face::sign_of_the_horns::thumbs_up:
mellow. dramatic!: I'm passing away
Liz Brauer: the new episode is great!
Flat Earth: yes but unique side chat every time about the video. Is always new. 
Juliana Elise 🦋: that's a fair way to look at it
sol cutta: this is a classic , I guess thats why ..its a important historic one isnt it.
Flat Earth: Heck, i think like an ai mod for the chat would be interesting - it could learn to propagate solutions i'de bet. 
sol cutta: theres supposed to be loads on chat already ..he did a video on that too...the dead internet
arf boucher: what ?
Marz Arnold: yeah thats a 

In [ ]:
from google.colab import files
files.download('yt_live_chat.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install pandas
!pip install torch
!pip install transformers
!pip install emoji
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 904.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re
from tqdm import tqdm

# ========== 1. Load Toxic-BERT ==========
MODEL_NAME = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.eval()

# ========== 2. Toxicity Labels ==========
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# ========== 3. Clean Function ==========
def clean_text(text):
    # Convert to string and handle potential NaN values
    text = str(text) if not pd.isna(text) else ""
    text = re.sub(r'[^\w\s]', '', text)                # Remove punctuation
    text = text.lower().strip()                        # Lowercase & trim
    return text

# ========== 4. Get Toxicity Scores ==========
def get_toxicity_scores(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits)[0].numpy()
    return {label: float(score) for label, score in zip(labels, probs)}

    # ========== 4 Load Sentiment Model ==========
sentiment_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name)
sentiment_model.eval()

# ========== 4. Sentiment Detection Function ==========
def get_sentiment_label(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        star = torch.argmax(probs, dim=1).item() + 1
        if star <= 2:
            return "Negative"
        elif star == 3:
            return "Neutral"
        else:
            return "Positive"


# ========== 5. Load CSV ==========
df = pd.read_csv("yt_live_chat.csv")  # ✅ Your file name

# Rename the column if needed
if 'comment' not in df.columns:
    df.rename(columns={df.columns[1]: 'comment'}, inplace=True)

df['cleaned_comment'] = df['comment'].apply(clean_text)

# ========== 6. Run Detection ==========
toxicity_data = []
print("Detecting toxicity on comments...")

for comment in tqdm(df['cleaned_comment']):
    scores = get_toxicity_scores(comment)
    toxicity_data.append(scores)

tox_df = pd.DataFrame(toxicity_data)

print("Classifying sentiment...")

# Add a progress bar while applying sentiment
df['sentiment'] = [get_sentiment_label(c) for c in tqdm(df['cleaned_comment'])]


# ========== 7. Combine with Original ==========
result_df = pd.concat([df, tox_df], axis=1)

# Flag as toxic if any score > 0.5
result_df['is_toxic'] = result_df[labels].max(axis=1) > 0.5

# ========== 8. Save to CSV ==========
result_df.to_csv("toxic_comments_labeled.csv", index=False)
print("✅ Saved to toxic_comments_labeled.csv")

Detecting toxicity on comments...


100%|██████████| 208/208 [00:23<00:00,  8.92it/s]


Classifying sentiment...


100%|██████████| 208/208 [00:20<00:00, 10.07it/s]

✅ Saved to toxic_comments_labeled.csv
